In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import torch

In [3]:
from datasets import load_dataset

data_down_dir = "/root/storage/nas/JH_server/2025/Synthetic_data/0_dataset"

dataset = load_dataset(f"{data_down_dir}/web_of_science")
# 예시 출력
print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['input_data', 'label', 'label_level_1', 'label_level_2'],
        num_rows: 11967
    })
})
{'input_data': 'The aim of this study was to investigate (a) the behavioral cues that are displayed by, and trait judgments formed about, anxious interviewees, and (b) why anxious interviewees receive lower interview performance ratings. The Behavioral Expression of Interview Anxiety Model was created as a conceptual framework to explore these relations. We videotaped and transcribed mock job interviews, obtained ratings of interview anxiety and interview performance, and trained raters to assess several verbal and nonverbal cues and trait judgments. The results indicated that few behavioral cues, but several traits were related to interviewee and interviewer ratings of interview anxiety. Two factors emerged from our factor analysis on the trait judgments-Assertiveness and Interpersonal Warmth. Mediation analyses were performed and indicated 

In [4]:
from collections import defaultdict

label_counter = defaultdict(int)

for row in dataset['train']:
    label_counter[row['label']] += 1

In [ ]:
from sklearn.model_selection import train_test_split


model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir='/root/storage/nas/JH_server/cache/')

def preprocess(inputs):
    return tokenizer(inputs['input_data'], truncation=True, padding="max_length", max_length=128)

split_dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

train_dataset = train_dataset.map(preprocess, batched=True)
test_dataset = test_dataset.map(preprocess, batched=True)




num_labels =  len(label_counter)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, cache_dir='/root/storage/nas/JH_server/cache/')

accuracy = evaluate.load("accuracy")

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy.compute(predictions=preds, references=labels)

    return acc

args = TrainingArguments(
    eval_strategy="steps",
    eval_steps=100,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    save_strategy="no"
)

trainer = Trainer(
    model = model,
    args=args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

                                    

Map:   0%|          | 0/9573 [00:00<?, ? examples/s]

Map:   0%|          | 0/2394 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_334237/810534724.py:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy
100,No log,1.362813,0.729741
200,No log,0.898646,0.776107
300,No log,0.779320,0.800334
400,No log,0.762500,0.801170
500,1.047900,0.787704,0.799081
600,1.047900,0.784873,0.794904
700,1.047900,0.824523,0.790309
800,1.047900,0.855407,0.799499
900,1.047900,0.856276,0.799081
1000,0.188100,0.870539,0.802423
